In [1]:
#!/usr/bin/env python
# coding: utf-8


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns;

sns.set()
get_ipython().run_line_magic('matplotlib', 'inline')

In [3]:
from keras.utils import plot_model
from keras.models import Sequential, load_model
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

Using TensorFlow backend.


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load The dataset

In [7]:
data_attack = pd.read_csv('../../dataset/dataset_attack.csv')

In [8]:
data_normal = pd.read_csv('../../dataset/dataset_normal.csv')

In [9]:
data_normal.columns=[ 'frame.len', 'frame.protocols', 'ip.hdr_len',
       'ip.len', 'ip.flags.rb', 'ip.flags.df', 'p.flags.mf', 'ip.frag_offset',
       'ip.ttl', 'ip.proto', 'ip.src', 'ip.dst', 'tcp.srcport', 'tcp.dstport',
       'tcp.len', 'tcp.ack', 'tcp.flags.res', 'tcp.flags.ns', 'tcp.flags.cwr',
       'tcp.flags.ecn', 'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push',
       'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size',
       'tcp.time_delta','class']
data_attack.columns=[ 'frame.len', 'frame.protocols', 'ip.hdr_len',
       'ip.len', 'ip.flags.rb', 'ip.flags.df', 'p.flags.mf', 'ip.frag_offset',
       'ip.ttl', 'ip.proto', 'ip.src', 'ip.dst', 'tcp.srcport', 'tcp.dstport',
       'tcp.len', 'tcp.ack', 'tcp.flags.res', 'tcp.flags.ns', 'tcp.flags.cwr',
       'tcp.flags.ecn', 'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push',
       'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size',
       'tcp.time_delta','class']

# Drop unwanted columns

In [10]:
data_normal=data_normal.drop(['ip.src', 'ip.dst','frame.protocols'],axis=1)
data_attack=data_attack.drop(['ip.src', 'ip.dst','frame.protocols'],axis=1)

In [11]:
features=[ 'frame.len', 'ip.hdr_len',
       'ip.len', 'ip.flags.rb', 'ip.flags.df', 'p.flags.mf', 'ip.frag_offset',
       'ip.ttl', 'ip.proto', 'tcp.srcport', 'tcp.dstport',
       'tcp.len', 'tcp.ack', 'tcp.flags.res', 'tcp.flags.ns', 'tcp.flags.cwr',
       'tcp.flags.ecn', 'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push',
       'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size',
       'tcp.time_delta']

In [12]:
X_normal= data_normal[features].values
X_attack= data_attack[features].values
Y_normal= data_normal['class']
Y_attack= data_attack['class']
X=np.concatenate((X_normal,X_attack))
Y=np.concatenate((Y_normal,Y_attack))

# Standardise the data

In [13]:
scalar = StandardScaler(copy=True, with_mean=True, with_std=True)
scalar.fit(X)
X = scalar.transform(X)

# If the class label is 'attack' replace it with a 0 and if the label is 'normal' use 1.

In [14]:
for i in range(0,len(Y)):
  if Y[i] =="attack":
    Y[i]=0
  else:
    Y[i]=1
    

# Split the dataset into train and test set

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [16]:
print(Y_train)

[0 0 1 ... 0 1 1]


Architecture used

![Model](model_ann.png)

# Create model and fit training data

In [17]:
def create_baseline():
    # create the model
    model = Sequential()
    model.add(Dense(8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model. We use the the logarithmic loss function, and the Adam gradient optimizer.
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [18]:
model = create_baseline()
# model = load_model('ann_model.h5')

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
history = model.fit(X_train, Y_train, validation_split = 0.2, epochs = 100)

Instructions for updating:
Use tf.cast instead.
Train on 3621758 samples, validate on 905440 samples
Epoch 1/100
3621758/3621758 [==============================] - 149s 41us/step - loss: 0.3546 - acc: 0.8245 - val_loss: 0.3097 - val_acc: 0.8655
Epoch 2/100
3621758/3621758 [==============================] - 149s 41us/step - loss: 0.3005 - acc: 0.8592 - val_loss: 0.2972 - val_acc: 0.8590
Epoch 3/100
3621758/3621758 [==============================] - 149s 41us/step - loss: 0.2930 - acc: 0.8622 - val_loss: 0.2898 - val_acc: 0.8674
Epoch 4/100
3621758/3621758 [==============================] - 149s 41us/step - loss: 0.2897 - acc: 0.8635 - val_loss: 0.2888 - val_acc: 0.8607
Epoch 5/100
3621758/3621758 [==============================] - 149s 41us/step - loss: 0.2886 - acc: 0.8640 - val_loss: 0.2880 - val_acc: 0.8654
Epoch 6/100
3621758/3621758 [==============================] - 149s 41us/step - loss: 0.2881 - acc: 0.8643 - val_loss: 0.2875 - val_acc: 0.8633
Epoch 7/100
3621758/3621758 [======

3621758/3621758 [==============================] - 149s 41us/step - loss: 0.2858 - acc: 0.8660 - val_loss: 0.2845 - val_acc: 0.8661
Epoch 56/100
3621758/3621758 [==============================] - 149s 41us/step - loss: 0.2859 - acc: 0.8661 - val_loss: 0.2870 - val_acc: 0.8666
Epoch 57/100
3621758/3621758 [==============================] - 149s 41us/step - loss: 0.2858 - acc: 0.8661 - val_loss: 0.2849 - val_acc: 0.8687
Epoch 58/100
3621758/3621758 [==============================] - 149s 41us/step - loss: 0.2858 - acc: 0.8661 - val_loss: 0.2840 - val_acc: 0.8667
Epoch 59/100
3621758/3621758 [==============================] - 149s 41us/step - loss: 0.2858 - acc: 0.8661 - val_loss: 0.2852 - val_acc: 0.8678
Epoch 60/100
3621758/3621758 [==============================] - 149s 41us/step - loss: 0.2858 - acc: 0.8661 - val_loss: 0.2842 - val_acc: 0.8682
Epoch 61/100
3621758/3621758 [==============================] - 149s 41us/step - loss: 0.2858 - acc: 0.8661 - val_loss: 0.2863 - val_acc: 0.865

# Plot training & validation accuracy values

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('ANN Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('ANN Model Accuracy.png')
plt.show()

# Plot training & validation loss values

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('ANN Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('ANN Model loss.png')
plt.show()

In [ ]:
predict = model.predict(X_test, verbose=1)

# Calculate True positive, True negetive, False positive and False negetive values and create a Heatmap

In [ ]:
tp = 0
tn = 0
fp = 0
fn = 0
predictn = predict.flatten().round()
predictn = predictn.tolist()
Y_testn = Y_test.tolist()
for i in range(len(Y_testn)):
  if predictn[i]==1 and Y_testn[i]==1:
    tp+=1
  elif predictn[i]==0 and Y_testn[i]==0:
    tn+=1
  elif predictn[i]==0 and Y_testn[i]==1:
    fp+=1
  elif predictn[i]==1 and Y_testn[i]==0:
    fn+=1


In [ ]:
to_heat_map =[[tn,fp],[fn,tp]]
to_heat_map = pd.DataFrame(to_heat_map, index = ["Attack","Normal"],columns = ["Attack","Normal"])
ax = sns.heatmap(to_heat_map,annot=True, fmt="d")

# Save details

In [ ]:
figure = ax.get_figure()    
figure.savefig('confusion_matrix_ANN.png', dpi=400)

In [ ]:
model.save('ann_model.h5')
plot_model(model, to_file='model_ann.png',show_shapes=True, show_layer_names=True)

In [ ]:
ann_viz(model, title="My first neural network")

In [ ]:
scores = model.evaluate(X_test, Y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))